📌 1. Imports

In [ ]:
# Regular expressions for parsing input lines
import re

# Dictionary with default values for nested transitions
from collections import defaultdict

# For drawing the DFA using Graphviz
import graphviz

# For uploading files in Google Colab
from google.colab import files

📌 2. DFA Class Definition and Initialization

In [ ]:
class DFA:
    def __init__(self, alphabet, states, initial, finals, transitions):
        # Basic DFA components
        self.alphabet = alphabet
        self.states = states
        self.initial = initial
        self.finals = finals
        self.transitions = transitions
